In [3]:
from sklearn.datasets import fetch_openml
import PIL
import numpy
from PIL import Image
import math
mnist = fetch_openml("mnist_784")

In [4]:
ss_width = 2048
image_height = 28
image_width = 28

In [5]:
columns = math.floor(ss_width/image_width)

In [63]:
all_rows = []
for i in range(len(mnist.data)):
    if i % columns == 0:
        all_rows.append([])
    current = all_rows[len(all_rows) - 1]
    current.append(mnist.data.loc[i].values)

In [74]:
mnist.data
mnist.target

0        5
1        0
2        4
3        1
4        9
        ..
69995    2
69996    3
69997    4
69998    5
69999    6
Name: class, Length: 70000, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

In [ ]:
mnist.data.loc[i].values

In [65]:
len(all_rows)

959

In [66]:
ranges = []
for i in range(math.ceil(len(all_rows) / columns)):
    start = i * columns
    end = (i + 1) * columns
    if i == math.ceil(len(all_rows) / columns) - 1:
        end = len(all_rows)
    ranged = range(start, end)
    ranges.append(ranged)

In [17]:
ranges

[range(0, 73),
 range(73, 146),
 range(146, 219),
 range(219, 292),
 range(292, 365),
 range(365, 438),
 range(438, 511),
 range(511, 584),
 range(584, 657),
 range(657, 730),
 range(730, 803),
 range(803, 876),
 range(876, 949),
 range(949, 959)]

In [67]:
leftover = 2048 % columns
row_padding = []
for i in range(leftover):
    row_padding.append(0)
row_padding

[0, 0, 0, 0]

In [68]:
chunk_padding = []
for i in range(2048 - columns * image_height):
    chunk_padding.append([])
    current = chunk_padding[len(chunk_padding) - 1]
    for p in range(2048):
        current.append(0)
len(chunk_padding)

4

In [69]:
len(all_rows[0])

73

In [70]:
ss_width - len(ranges[0]) * image_height

4

In [72]:
import numpy as np
import array
from PIL import Image

for range_counter in range(len(ranges)):
    full_combined = []
    for r in ranges[range_counter]:
        row_digits = all_rows[r]
        reshaped = []
        for i in range(len(row_digits)):
            digit = row_digits[i]
            digit = np.resize(digit,(image_width,image_height))
            reshaped.append(digit)
        combined = []
        for i in range(image_height):
            combined.append([])
            current = combined[len(combined) - 1]
            for j in range(len(row_digits)):
                current.extend(reshaped[j][i])
            padding_needed = ss_width - len(row_digits) * image_width
            for p in range(padding_needed):
                current.append(0)
        full_combined.extend(combined)
    row_padding_needed = ss_width - len(full_combined)
    wtf = row_padding_needed
    for rp in range(row_padding_needed):
        current = []
        for p in range(ss_width):
            current.append(0)
        full_combined.append(current)
    test = full_combined
    rgba_combined = []
    for row in full_combined:
        rgba_combined.append([])
        current = rgba_combined[len(rgba_combined) - 1]
        for item in row:
            current.append((item, item, item, 255))
    rgba_combined = np.asarray(rgba_combined)
    rgba_combined = rgba_combined.astype(np.uint8)
    im = Image.fromarray(np.asarray(rgba_combined), mode='RGBA')
    name = 'mnist_tile_solid_' + str(range_counter) + '.png'
    im.save(name)

In [71]:
for range_counter in range(len(ranges)):
    full_combined = []
    for r in ranges[range_counter]:
        row_digits = all_rows[r]
        reshaped = [np.resize(d, (image_width, image_height)) for d in row_digits]
        combined = [np.concatenate(reshaped, axis=1)]
        padding_needed = ss_width - combined[0].shape[1]
        combined[0] = np.pad(combined[0], ((0, 0), (0, padding_needed)), mode='constant', constant_values=0)
        full_combined.extend(combined)
    padding_needed = ss_width - full_combined[-1].shape[1]
    full_combined[-1] = np.pad(full_combined[-1], ((0, 0), (0, padding_needed)), mode='constant', constant_values=0)
    rgba_combined = np.repeat(np.expand_dims(full_combined, axis=2), repeats=4, axis=2)
    rgba_combined[..., :-1] = rgba_combined[..., 0:1]
    im = Image.fromarray(np.uint8(rgba_combined))
    name = f'mnist_tile_solid_{range_counter}.png'
    im.save(name)

TypeError: Cannot handle this data type: (1, 1, 4, 2048), |u1

In [53]:

for range_counter in range(len(ranges)):
    i = 0
    full_combined = []
    for r in ranges[range_counter]:
        print(r)
        row_digits = all_rows[r]
        if i > 2:
            break
# import matplotlib.pyplot as plt

# plt.imshow(full_combined)
# plt.show()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
mnist.target[18378]

2.0

In [43]:
check = mnist.data[67620]
check = numpy.asarray(check)
check.resize(28,28)
im = Image.fromarray(check, mode='L')
im

KeyError: 67620